R&E Model 설계용 ipynb

Summary<br>
semi-supervised learning 기법을 이용하여 SNS Text Sentiment Analysis의 정확도를 높이는 방법을 연구하였다. 다양한 주제의 유튜브 영상 댓글, 대댓글과 Korean Hate Speech data set을 학습에 이용했다. Teacher model인 KcELECTRA(user generated, noisy text에서 높은 성능을 보임)를 통해 SNS Text의 특성을 고려한 column, domain정보에 관한 column, sentiment analysis 결과(긍정, 중립, 부정)를 나타내는 column으로 pseudo-labeling을 진행해 pseudo-label이 붙은 data를 만들었다. 이를 이용해 GPT-3, Clova sentiment, KcELECTRA, 3개의 Student model을 self-training 형식으로 학습했다. in-domain data(domain data)를 고려한 모델과 그렇지 않은 모델로 나누었고, 실험 시 전체 또는 일부의 column(in-domain data 제외)을 선택하여 모델을 훈련했다. SNS Text feature를 고려했을 때, SNS Text feature, domain data를 같이 고려했을 때 훈련시킨 모델과 KcELECTRA의 학습속도, acc 등을 비교했다. 가장 좋은 결과가 나온 Student model의 column을 바탕으로 SNS Text Sentiment Analysis 모델을 설계 및 개선하였다.

Check_list for running
1. labeled xlsx file 
2. Gdrive file & folder path
3. API key

colab의 For_net_file_label에서 코드 확인<br>1. test set 나누기 및 train set과 분리<br>2. null 확인<br>3. 개수 확인<br>4. preprocessing data 확인

Mounting

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Cloning

In [ ]:
!git clone https://github.com/shreyashankar/gpt3-sandbox.git
!cd gpt3-sandbox

Cloning into 'gpt3-sandbox'...
remote: Enumerating objects: 2471, done.
remote: Counting objects: 100% (67/67), done.
remote: Compressing objects: 100% (66/66), done.
remote: Total 2471 (delta 31), reused 7 (delta 1), pack-reused 2404
Receiving objects: 100% (2471/2471), 5.47 MiB | 17.68 MiB/s, done.
Resolving deltas: 100% (698/698), done.


install library

In [ ]:
!pip install google-api-python-client
!pip install seaborn
!pip install soynlp
!pip install openai
!pip install api
!pip install Transformers
!pip install emoji
!pip install pytorch_lightning
!pip install --upgrade tensorflow
!pip install delayed
!pip install --upgrade scikit-learn

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 416 kB 4.8 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 43 kB 1.5 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 163 kB 8.6 MB/s 
  Created wheel for openai: filename=openai-0.23.0-py3-none-any.whl size=54478 sha256=8c1b09af7174c63dbfe7429a6f28f06da8bfc61efa9f5c1d76800ab2b881d7ad
  Stored in directory: /root/.cache/pip/wheels/70/d5/31/f9f67660319d89e4f54501d27b1e90f88a3309c42ea4fd734c
Successfully built openai
Looking in indexes: http

In [ ]:
!pip uninstall pandas
!pip install pandas==1.1.5

Found existing installation: pandas 1.3.5
Uninstalling pandas-1.3.5:
  Would remove:
    /usr/local/lib/python3.7/dist-packages/pandas-1.3.5.dist-info/*
    /usr/local/lib/python3.7/dist-packages/pandas/*
  Would not remove (might be manually added):
    /usr/local/lib/python3.7/dist-packages/pandas/__init__.pyi
    /usr/local/lib/python3.7/dist-packages/pandas/_config/__init__.pyi
    /usr/local/lib/python3.7/dist-packages/pandas/_config/config.pyi
    /usr/local/lib/python3.7/dist-packages/pandas/_config/dates.pyi
    /usr/local/lib/python3.7/dist-packages/pandas/_config/display.pyi
    /usr/local/lib/python3.7/dist-packages/pandas/_config/localization.pyi
    /usr/local/lib/python3.7/dist-packages/pandas/_libs/__init__.pyi
    /usr/local/lib/python3.7/dist-packages/pandas/_libs/indexing.pyi
    /usr/local/lib/python3.7/dist-packages/pandas/_libs/interval.pyi
    /usr/local/lib/python3.7/dist-packages/pandas/_libs/missing.pyi
    /usr/local/lib/python3.7/dist-packages/pandas/_libs/pr

In [ ]:
#!pip install git+https://github.com/huggingface/transformers.git

Setting check

In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Thu Dec  2 02:55:17 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.44       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   36C    P0    25W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

Your runtime has 13.6 gigabytes of available RAM

Not using a high-RAM runtime


import library

In [ ]:
# Common
import pandas as pd
import numpy as np
import json
%matplotlib inline
import matplotlib.pyplot as plt
from io import StringIO
import sys
import os
from pprint import pprint
import webbrowser
import sklearn
import csv
from glob import glob
import time
from shutil import copyfile

# Text Preprocessing
import emoji
import re
from soynlp.normalizer import *
from collections import Counter
import string

# Text Collecting
from googleapiclient.discovery import build

#Modeling
import openai
import requests
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, TensorDataset
from torch.optim.lr_scheduler import ExponentialLR
from pytorch_lightning import LightningModule, Trainer, seed_everything
from transformers import AutoModelForSequenceClassification, AutoTokenizer, AdamW
from transformers import GPT2TokenizerFast
import tensorflow as tf

#Training & Testing
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from pytorch_lightning import loggers as my_logger
import datetime
from torch.utils.tensorboard import SummaryWriter

File provider<br>*오류 시 Merge_csv_file.ipynb file 참고

In [ ]:
folder_path = 'type your gdrive folder path that you stored data file (*list)'
merge_path = 'type your gdrive file path that you want store merged file (*list)'

for folder, merge_file in zip(folder_path, merge_path):
    file_list = glob.glob(path + '*')
    sum = 0
    with open(merge_file, 'w') as f:
        for i, file in enumerate(file_list):
            if i==0:
                with open(file, 'r') as f2:
                    while True:
                        line = f2.readline()

                        if not line:
                            break

                        f.write(line)

                file_name = file.split('\\')[-1]
                print(file.split('\\')[-1] + ' write compelete...!')
            else:
                with open(file, 'r') as f2:
                    n=0

                    while True:
                        line = f2.readline()

                        if n != 0:
                            f.write(line)
                        
                        if not line:
                            break

                        n += 1

                file_name = file.split('\\')[-1]
                print(file.split('\\')[-1] + ' write complete...!')
                print('number of line ', n)
                sum += n
    print('>>> ALL file merge complete...!')
    print('number of line ', sum)

In [ ]:
net_file_name = ['type file name you want(csv)']
labeling_file_name = ['type file name you want(for labeling, csv)']

for net_file in merge_path:
    df = pd.read_csv(net_file)
    df = df.drop_duplicates()
    df = sklearn.utils.shuffle(df)
    print('number of NAN ', df.isnull().sum())
    print('Shape ', df.shape)

    df.to_csv(net_file_name, index=None))
    df_1 = df[:1000]
    df_1.to_csv(labeling_file_name, index=None)

In [ ]:
#after labeling
#check your file according to the id 
#id = ['/content/V&&C_label.xlsx', '/content/O_label.xlsx', '/content/S_label.xlsx', '/content/KHS_net_label.xlsx']
id = ['/content/KHS_net_label.xlsx']

for i in id:

    input_name=i
    output_name=input_name.split('.xlsx')[0]+'.csv'
    xlsx = pd.read_excel(input_name)
    xlsx.to_csv(output_name, index=None)

#for KHS
df_be = pd.read_csv('/content/KHS_net_label.csv')
df_a = pd.read_csv('/content/for_KHS_net_label.tsv', sep='\t')
anti = ~((df_a['contain_gender_bias']==False) & (df_a['bias']=='none') & (df_a['hate']=='none'))

df_be = df_be[anti]
df_be = df_be.dropna(how='any')
df_be['comment'] = df_be['comment'].replace("[^ㄱ-ㅎㅏ-ㅣ가-힣]","")
df_be['comment'].replace('', np.nan)
df_be = df_be.dropna(how='any')

df_be.to_csv('KHS_label.csv')
print(df_be.isnull().sum())
df_be

comment      0
sentiment    0
dtype: int64


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:18: UserWarning: Boolean Series key will be reindexed to match DataFrame index.


,comment,sentiment
0,현재호텔주인심정아난마른하늘에날벼락맞고호텔망하게생겼는데누군계속추모받네,-1
2,못된넘들남의고통을즐겼던넘들이젠마땅한처벌을받아야지그래야공정한사회지심은대로거두거라,-1
4,사람얼굴손톱으로긁은것은인격살해이고동영상이몰카냐메걸리안들생각이없노,-1
6,년안에남녀간성전쟁한번크게치룬후일부다처제여성의정치참여금지여성투표권삭제가세계의공통문화로...,-1
8,년만에재미를느끼는프로였는데왜니들때문에폐지를해야되냐,-1
...,...,...
7854,히예전에나왔던히든싱어보면송혜교닮았음ㅋ,0
7859,힘내라힘꽃길만걷자준희양엄마가하늘에서보고계시니꼬옥힘내라,1
7862,힘내세요반드시더건강하고이쁜천사가찾아와요저도아가낳기전에한번의아픔을겪었는데솔직히시간이약...,1
7863,힘내세요한고은님아래한남충글뭐냐풉평생연애하나못하고홀애비로살다가라니같은건여자랑결혼할자격읍다,1


Declaration of Variables

In [ ]:
s = '뭐야 이 평점들은.... 나쁘진 않지만 10점 짜리는 더더욱 아니잖아	'
s = s.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣]","")
s

'뭐야 이 평점들은.... 나쁘진 않지만 10점 짜리는 더더욱 아니잖아\t'

In [ ]:
youtube_api_key=''

naver_client_id=''
naver_client_secret=''
naver_api_url="https://naveropenapi.apigw.ntruss.com/sentiment-analysis/v1/analyze"

naver_headers = {
    "X-NCP-APIGW-API-KEY-ID": naver_client_id,
    "X-NCP-APIGW-API-KEY": naver_client_secret,
    "Content-Type": "application/json"
}

openai_api_key=''

Data collector Youtube_API / 다른 방법(Twitter의 경우 많은 필터링 필요) 
youtube 인기동영상의 경우 더욱 많은 필터링이 자체적으로 이루어짐.
SNS 문장의 특성에 관해 다룬 논문을 참고할 필요가 있으며 이에 따라 적절한 동영상 댓글 수집 필요 
Domained data를 만들기 위한 
영상 선택도 필요


Data Collector

In [ ]:
cnt = 0
videos_ids = pd.read_excel('/content/Domain_data_num.xlsx')
videos_ids.drop(columns=['Subject', 'V&&C', 'N', 'S', 'E ', 'M', 'H', 'O',
       'Unnamed: 9', 'Unnamed: 10', 'Unnamed: 11', 'Unnamed: 12',
       'Unnamed: 13', 'Unnamed: 14', 'Unnamed: 15'], inplace=True)
videos_ids.dropna(how='any', inplace=True)

for video_id in videos_ids['Video_ID']:
    time.sleep(60)
    comments = list()
    api_obj = build('youtube', 'v3', developerKey=youtube_api_key)
    response = api_obj.commentThreads().list(part='snippet,replies', videoId=video_id, maxResults=100, textFormat='plainText').execute()

    while response:
        for item in response['items']:
            comment = item['snippet']['topLevelComment']['snippet']
            comments.append([comment['textOriginal'], comment['publishedAt'], comment['likeCount']])

        if 'nextPageToken' in response:
            response = api_obj.commentThreads().list(part='snippet,replies', videoId=video_id, pageToken=response['nextPageToken'], maxResults=100).execute()
        else:
            break

        print(comment)
    cnt += 1 
    df = pd.DataFrame(comments)

    if cnt <= 33:
        i = 'C_'+str(cnt)+'.csv'
    elif cnt > 33 and cnt <= 103:
        i = 'E_'+str(cnt-33)+'.csv'
    elif cnt > 103 and cnt <= 158:
        i = 'M_'+str(cnt-103)+'.csv'
    elif cnt > 158 and cnt <= 221:
        i = 'N_'+str(cnt-158)+'.csv'
    elif cnt > 221 and cnt <= 282:
        i = 'O_'+str(cnt-221)+'.csv'
    elif cnt > 282 and cnt <= 346:
        i = 'S_'+str(cnt-282)+'.csv'
    else:
        i = 'V_'+str(cnt-346)+'.csv'

    df.to_csv(i, header=['comment', 'date', 'num_likes'], index=None, encoding='utf-8')

In [ ]:
if not os.path.exists('./nsmc'):
    !git clone https://github.com/e9t/nsmc

Cloning into 'nsmc'...
remote: Enumerating objects: 14763, done.
remote: Total 14763 (delta 0), reused 0 (delta 0), pack-reused 14763
Receiving objects: 100% (14763/14763), 56.19 MiB | 22.38 MiB/s, done.
Resolving deltas: 100% (1749/1749), done.
Checking out files: 100% (14737/14737), done.


In [ ]:
!head nsmc/ratings_train.txt

id	document	label
9976970	아 더빙.. 진짜 짜증나네요 목소리	0
3819312	흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나	1
10265843	너무재밓었다그래서보는것을추천한다	0
9045019	교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정	0
6483659	사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 던스트가 너무나도 이뻐보였다	1
5403919	막 걸음마 뗀 3세부터 초등학교 1학년생인 8살용영화.ㅋㅋㅋ...별반개도 아까움.	0
7797314	원작의 긴장감을 제대로 살려내지못했다.	0
9443947	별 반개도 아깝다 욕나온다 이응경 길용우 연기생활이몇년인지..정말 발로해도 그것보단 낫겟다 납치.감금만반복반복..이드라마는 가족도없다 연기못하는사람만모엿네	0
7156791	액션이 없는데도 재미 있는 몇안되는 영화	1


In [ ]:
df_be = pd.read_csv('/content/nsmc/ratings_test.txt', sep='\t')
df_be.rename(columns={'document':'comment', 'label':'sentiment'}, inplace=True)
df_be

,id,comment,sentiment
0,6270596,굳 ㅋ,1
1,9274899,GDNTOPCLASSINTHECLUB,0
2,8544678,뭐야 이 평점들은.... 나쁘진 않지만 10점 짜리는 더더욱 아니잖아,0
3,6825595,지루하지는 않은데 완전 막장임... 돈주고 보기에는....,0
4,6723715,3D만 아니었어도 별 다섯 개 줬을텐데.. 왜 3D로 나와서 제 심기를 불편하게 하죠??,0
...,...,...,...
49995,4608761,오랜만에 평점 로긴했네ㅋㅋ 킹왕짱 쌈뽕한 영화를 만났습니다 강렬하게 육쾌함,1
49996,5308387,의지 박약들이나 하는거다 탈영은 일단 주인공 김대희 닮았고 이등병 찐따 OOOO,0
49997,9072549,그림도 좋고 완성도도 높았지만... 보는 내내 불안하게 만든다,0
49998,5802125,절대 봐서는 안 될 영화.. 재미도 없고 기분만 잡치고.. 한 세트장에서 다 해먹네,0


In [ ]:
pd.options.mode.chained_assignment = 'warn'
cnt = 0
characters = "'.[0-9]#\:"
columns = ['comment']

s = pd.DataFrame(columns=columns)

for i in df_be['comment']:
    if type(i) == str:
        if len(i) < 60 and len(i) > 5:
            for x in range(len(characters)):
                q = characters[x]
                r = i.replace(q, "")

            r = r.strip()
            r = emoticon_normalize(r, num_repeats=2)
            comment = r
            row = [comment]
            series = pd.Series(row, index=s.columns)
            s = s.append(series, ignore_index=True)

s['comment'] = s['comment'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","") #숫자, !, ? 등 포함시켜서도 해보기
s['comment'].replace('', np.nan)

s = s.dropna(how='any')

In [ ]:
df_be['comment'] = df_be['comment'].replace("[^ㄱ-ㅎㅏ-ㅣ가-힣]","")
df_be['comment'].replace('', np.nan)
#df_be.dropna(how='any', inplace=True, axis=0)

df_be.to_csv('/content/ratings_test.txt', index=False, sep='\t')
print(df_be.isnull().sum())
df_be

id           0
comment      3
sentiment    0
dtype: int64


,id,comment,sentiment
0,6270596,굳 ㅋ,1
1,9274899,GDNTOPCLASSINTHECLUB,0
2,8544678,뭐야 이 평점들은.... 나쁘진 않지만 10점 짜리는 더더욱 아니잖아,0
3,6825595,지루하지는 않은데 완전 막장임... 돈주고 보기에는....,0
4,6723715,3D만 아니었어도 별 다섯 개 줬을텐데.. 왜 3D로 나와서 제 심기를 불편하게 하죠??,0
...,...,...,...
49995,4608761,오랜만에 평점 로긴했네ㅋㅋ 킹왕짱 쌈뽕한 영화를 만났습니다 강렬하게 육쾌함,1
49996,5308387,의지 박약들이나 하는거다 탈영은 일단 주인공 김대희 닮았고 이등병 찐따 OOOO,0
49997,9072549,그림도 좋고 완성도도 높았지만... 보는 내내 불안하게 만든다,0
49998,5802125,절대 봐서는 안 될 영화.. 재미도 없고 기분만 잡치고.. 한 세트장에서 다 해먹네,0


In [ ]:
df_be = pd.read_csv('/content/nsmc/ratings_train.txt', sep='\t')
df_be.rename(columns={'document':'comment', 'label':'sentiment'}, inplace=True)
df_be

,id,comment,sentiment
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정,0
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...,1
...,...,...,...
149995,6222902,인간이 문제지.. 소는 뭔죄인가..,0
149996,8549745,평점이 너무 낮아서...,1
149997,9311800,이게 뭐요? 한국인은 거들먹거리고 필리핀 혼혈은 착하다?,0
149998,2376369,청춘 영화의 최고봉.방황과 우울했던 날들의 자화상,1


In [ ]:
pd.options.mode.chained_assignment = 'warn'
cnt = 0
characters = "'.[0-9]#\:"
columns = ['comment']

s = pd.DataFrame(columns=columns)

for i in df_be['comment']:
    if type(i) == str:
        if len(i) < 60 and len(i) > 5:
            for x in range(len(characters)):
                q = characters[x]
                r = i.replace(q, "")

            r = r.strip()
            r = emoticon_normalize(r, num_repeats=2)
            comment = r
            row = [comment]
            series = pd.Series(row, index=s.columns)
            s = s.append(series, ignore_index=True)

s['comment'] = s['comment'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","") #숫자, !, ? 등 포함시켜서도 해보기
s['comment'].replace('', np.nan)

s = s.dropna(how='any')

In [ ]:
df_be['comment'] = df_be['comment'].replace("[^ㄱ-ㅎㅏ-ㅣ가-힣]","")
df_be['comment'].replace('', np.nan)
#df_be.dropna(how='any', inplace=True, axis=0)

df_be.to_csv('/content/ratings_train.txt', index=False, sep='\t')
print(df_be.isnull().sum())
df_be

id           0
comment      5
sentiment    0
dtype: int64


,id,comment,sentiment
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정,0
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...,1
...,...,...,...
149995,6222902,인간이 문제지.. 소는 뭔죄인가..,0
149996,8549745,평점이 너무 낮아서...,1
149997,9311800,이게 뭐요? 한국인은 거들먹거리고 필리핀 혼혈은 착하다?,0
149998,2376369,청춘 영화의 최고봉.방황과 우울했던 날들의 자화상,1


Data preprocessing

soynlp에서 문법교정(띄어쓰기) 라이브러리 사용 고려
https://github.com/lovit/soyspacing / vectorize 할 때 성능 향상
/ 국립 국어원 말뭉치 활용

1. 문장 글자 수 조절
2. ㅋㅋㅋㅋㅋㅋㅋㅋㅋ -> ㅋㅋ
3. '#\/ 등의 문자열 제거
4. 영어 제거
5. 중복된 데이터 제거 
6. NaN값 제거
7. '{space}정보 R&E?{space}' -> '정보 R&E?'


In [ ]:
data = pd.read_csv('/content/results.csv', engine='python', error_bad_lines=False) #engine='python' 사용시 [CParserError: Error tokenizing data. C error: Buffer overflow caught - possible malformed input file.] 발생안함 / 2021.10.19
data    

                                                    #engine='c'를 사용하는 경우도 있음. / 2021.10.19
#error_bad_lines=False => engine='c' => error && engine='python' => error 해결가능(NULL Byte) / 2021.10.19

,comment,date,num_likes
0,판매 페이지를보니 아이패드 12.9는 앞부분만 전용 모델을 따로 구입해야될 것 같습...,2021-03-20T12:12:03Z,900
1,인트로 역대급인데 ㅋㅋㅋㅋㅋㅋㅋ,2021-11-12T16:23:22Z,0
2,1:20 아가씨 아닌가 ㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋ 여윽시...스킹...,2021-11-09T17:32:27Z,0
3,오옷 200만,2021-11-08T14:31:57Z,0
4,저 침대 거치대.. 약간 모서리가 찍힌 아이폰에 썼다가 아이폰 빠지직 하고 후면 다...,2021-11-03T07:12:16Z,0
...,...,...,...
1392,각선미 무쳤다,2021-03-20T12:01:34Z,3
1393,ㅗㅜㅑ...,2021-03-20T12:01:30Z,0
1394,어찌 인트로가 점점 병맛이 됩니까?????\n1빠,2021-03-20T12:01:13Z,4
1395,🥰,2021-03-20T12:00:59Z,0


In [ ]:
def below_threshold_len(max_len, nested_list):
    cnt = 0
    for s in nested_list:
        if(len(s) <= max_len):
            cnt += 1
    print('전체 샘플 중 길이가 %d이하인 샘플의 비율 : %f'%(max_len, (cnt / len(data))*100))

def upper_threshold_len(min_len, nested_list):
    cnt = 0
    for s in nested_list:
        if(len(s) >= min_len):
            cnt += 1
    print('전체 샘플 중 길이가 %d이상인 샘플의 비율 : %f'%(min_len, (cnt / len(data))*100))

In [ ]:
# max_len = 5
# min_len = 60
# text = data['comment']
# below_threshold_len(max_len, text)
# upper_threshold_len(min_len, text)

In [ ]:
pd.options.mode.chained_assignment = 'warn'
cnt = 0
characters = "'.[0-9]#\:"
columns = ['comment']

s = pd.DataFrame(columns=columns)

for i in data['comment']:
    if type(i) == str:
        if len(i) < 60 and len(i) > 5:
            for x in range(len(characters)):
                q = characters[x]
                r = i.replace(q, "")

            r = r.strip()
            r = emoticon_normalize(r, num_repeats=2)
            comment = r
            row = [comment]
            series = pd.Series(row, index=s.columns)
            s = s.append(series, ignore_index=True)

s['comment'] = s['comment'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","") #숫자, !, ? 등 포함시켜서도 해보기
s['comment'].replace('', np.nan)

s = s.dropna(how='any')

In [ ]:
print(s.isnull().sum())
s.to_csv('for_comment_only.csv', header=['comment'], index=None, encoding='utf-8')
for_Text = pd.read_csv('/content/for_comment_only.csv')
for_Text = for_Text.dropna(how='any')
for_Text = for_Text.drop_duplicates()
for_Text.to_csv('comment_only.csv', header=['comment'], index=None, encoding='utf-8')

comment    0
dtype: int64


In [ ]:
Text = pd.read_csv('/content/comment_only.csv')
print(Text.isnull().sum())

comment    0
dtype: int64


TEACHER MODEL

KcELECTRA

KcELECTRA model에서 사용한 학습 설계 방법 참고 / R&E 보고서 작성시 아래 코드 score 매기는 방법 활용하기

In [ ]:
id = ['/content/E_label',
      '/content/H_label',
      '/content/M_label',
      '/content/N_label',
      '/content/O_label',
      '/content/S_label',
      '/content/V&&C_label'
      ]
for i in id:

    input_name=i+'.xlsx'
    output_name=input_name.split('.xlsx')[0]+'.csv'
    xlsx = pd.read_excel(input_name)
    xlsx.to_csv(output_name, index=None)

    df = pd.read_csv(i+'.csv')
    cnt = 0
    for j in df['sentiment']:
        if type(df['sentiment'][cnt]) == str:
            if df['sentiment'][cnt] == str(0):
                df.iloc[cnt, df.columns.get_loc('sentiment')] = int(0)
            elif df['sentiment'][cnt] == str(1):
                df.iloc[cnt, df.columns.get_loc('sentiment')] = int(1)
            elif df['sentiment'][cnt] == str(2):
                df.iloc[cnt, df.columns.get_loc('sentiment')] = int(2)

        cnt += 1

    cnt = 0
    err = 0
    che = 0
    for j in df['sentiment']:
        if df['sentiment'][cnt] == str(0):
            err += 1
        elif df['sentiment'][cnt] == str(1):
            err += 1
        elif df['sentiment'][cnt] == str(2):
            err += 1
        else:
            che += 1

        cnt += 1

    print(err)
    print(che == cnt)

    print(df.isnull().sum())
    df.dropna(inplace=True)
    print('////////////////')
    print(df.isnull().sum())

    df.to_csv(i+'.csv', index=None)

    df_train = df.iloc[:800]
    df_val = df.iloc[800:900]
    df_test = df.iloc[900:1000]

    df_train.to_csv(i+'_train'+'.csv', index=None)
    df_val.to_csv(i+'_val'+'.csv', index=None)
    df_test.to_csv(i+'_test'+'.csv', index=None)

    print('------'+i+'------')

In [ ]:
args = {
    'random_seed': 42, # Random Seed
    'pretrained_model': 'beomi/KcELECTRA-base',  # Transformers PLM name
    'pretrained_tokenizer': '',  # Optional, Transformers Tokenizer Name. Overrides `pretrained_model`
    'batch_size': 32,
    'lr': 5e-6,  # Starting Learning Rate
    'epochs': 10,  # Max Epochs
    'max_length': 150,  # Max Length input size
    'train_data_path': "/content/Common_label_train.csv",  # Train Dataset file 
    'val_data_path': "/content/Common_label_val.csv",  # Validation Dataset file 
    'test_mode': False,  # Test Mode enables `fast_dev_run`
    'optimizer': 'AdamW',  # AdamW vs AdamP
    'lr_scheduler': 'exp',  # ExponentialLR vs CosineAnnealingWarmRestarts
    'fp16': True,  # Enable train on FP16(if GPU)
    'tpu_cores': 0,  # Enable TPU with 1 core or 8 cores
    'cpu_workers': os.cpu_count(),
}

epochs가 10 정도 되면 과적합 발생함.

In [ ]:
args

{'batch_size': 32,
 'cpu_workers': 2,
 'epochs': 10,
 'fp16': True,
 'lr': 5e-06,
 'lr_scheduler': 'exp',
 'max_length': 150,
 'optimizer': 'AdamW',
 'pretrained_model': 'beomi/KcELECTRA-base',
 'pretrained_tokenizer': '',
 'random_seed': 42,
 'test_mode': False,
 'tpu_cores': 0,
 'train_data_path': '/content/Common_label_train.csv',
 'val_data_path': '/content/Common_label_val.csv'}

In [ ]:
!nvidia-smi

Thu Dec  2 04:28:41 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.44       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   35C    P0    26W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

import cell에서 이미 선언한 LightningModule을 상속해서 Model에서 조금 변경하는 형식으로 코드 이루어짐<br>따라서 pytorch lightning module과 관련된 튜토리얼 document나<br>https://pytorch-lightning.readthedocs.io/en/stable/extensions/logging.html<br>다른 여러 사이트 참고해서 변형하기

In [ ]:
# tb_logger = my_logger.TensorBoardLogger('logs/')

In [ ]:
class Model(LightningModule):
    def __init__(self, **kwargs):
        super().__init__() 
        self.save_hyperparameters() # 이 부분에서 self.hparams에 위 kwargs가 저장된다.
        
        self.clsfier = AutoModelForSequenceClassification.from_pretrained(self.hparams.pretrained_model)
        #위의 clsfier 구조 뜯어보기 / 역전파 분석, 아마도 loss를 변형시키면 될 듯

        self.tokenizer = AutoTokenizer.from_pretrained(
            self.hparams.pretrained_tokenizer
            if self.hparams.pretrained_tokenizer
            else self.hparams.pretrained_model
        )
        self.time = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")

    def forward(self, **kwargs):
        return self.clsfier(**kwargs)

    def step(self, batch, batch_idx, state):
        data, labels = batch 
        #labels => 32 길이의 list == batch_size

        output = self(input_ids=data, labels=labels)
        # loss_for_plot = criterion(output, labels)
        # Transformers 4.0.0+
        loss = output.loss
        writer.add_scalar(state+'_loss'+self.time, loss, batch_idx)
        logits = output.logits

        preds = logits.argmax(dim=-1)

        y_true = list(labels.cpu().numpy())
        y_pred = list(preds.cpu().numpy())

        acc = accuracy_score(y_true, y_pred)
        prec = precision_score(y_true, y_pred)
        rec = recall_score(y_true, y_pred)
        f1 = f1_score(y_true, y_pred)

        writer.add_scalar(state+'_acc'+self.time, acc, batch_idx)
        writer.add_scalar(state+'_prec'+self.time, prec, batch_idx)
        writer.add_scalar(state+'_rec'+self.time, rec, batch_idx)
        writer.add_scalar(state+'_f1'+self.time, f1, batch_idx)

        return {
            'loss': loss,
            'y_true': y_true,
            'y_pred': y_pred,
        }

    def training_step(self, batch, batch_idx):
        return self.step(batch, batch_idx, 'training')

    def validation_step(self, batch, batch_idx):
        return self.step(batch, batch_idx, 'validation')

    def epoch_end(self, outputs, state='train'):
        loss = torch.tensor(0, dtype=torch.float)
        for i in outputs:
            loss += i['loss'].cpu().detach()
        loss = loss / len(outputs)

        y_true = []
        y_pred = []
        for i in outputs:
            y_true += i['y_true']
            y_pred += i['y_pred']
        
        acc = accuracy_score(y_true, y_pred)
        #만든 y_true배열에 i(output)에 들어있던 값을 넣은 후 y_pred와 비교 
        
        prec = precision_score(y_true, y_pred)
        rec = recall_score(y_true, y_pred)
        f1 = f1_score(y_true, y_pred)

        self.log(state+'_loss', float(loss), on_epoch=True, prog_bar=True)
        self.log(state+'_acc', acc, on_epoch=True, prog_bar=True)
        self.log(state+'_precision', prec, on_epoch=True, prog_bar=True)
        self.log(state+'_recall', rec, on_epoch=True, prog_bar=True)
        self.log(state+'_f1', f1, on_epoch=True, prog_bar=True)
        print(f'[Epoch {self.trainer.current_epoch} {state.upper()}] Loss: {loss}, Acc: {acc}, Prec: {prec}, Rec: {rec}, F1: {f1}')
        return {'loss': loss}
    
    def training_epoch_end(self, outputs):
        self.epoch_end(outputs, state='train')

    def validation_epoch_end(self, outputs):
        self.epoch_end(outputs, state='val')

    def configure_optimizers(self):
        if self.hparams.optimizer == 'AdamW':
            optimizer = AdamW(self.parameters(), lr=self.hparams.lr)
        #해당하는 파라미터와 학습률을 바탕으로 optimizer 설계

        elif self.hparams.optimizer == 'AdamP':
            from adamp import AdamP
            optimizer = AdamP(self.parameters(), lr=self.hparams.lr)
        else:
            raise NotImplementedError('Only AdamW and AdamP is Supported!')
        if self.hparams.lr_scheduler == 'cos':
            scheduler = CosineAnnealingWarmRestarts(optimizer, T_0=1, T_mult=2)
        #Scheduling을 통해 에포크 당 학습률 설정

        elif self.hparams.lr_scheduler == 'exp':
            scheduler = ExponentialLR(optimizer, gamma=0.5)
        else:
            raise NotImplementedError('Only cos and exp lr scheduler is Supported!')
        return {
            'optimizer': optimizer,
            'scheduler': scheduler,
        }

    def read_data(self, path):
        if path.endswith('xlsx'):
            return pd.read_excel(path)
        elif path.endswith('csv'):
            return pd.read_csv(path)
        elif path.endswith('tsv') or path.endswith('txt'):
            return pd.read_csv(path, sep='\t')
        else:
            raise NotImplementedError('Only Excel(xlsx)/Csv/Tsv(txt) are Supported')

    def clean(self, x):
        emojis = ''.join(emoji.UNICODE_EMOJI.keys())
        pattern = re.compile(f'[^ .,?!/@$%~％·∼()\x00-\x7Fㄱ-힣{emojis}]+')
        url_pattern = re.compile(
            r'https?:\/\/(www\.)?[-a-zA-Z0-9@:%._\+~#=]{1,256}\.[a-zA-Z0-9()]{1,6}\b([-a-zA-Z0-9()@:%_\+.~#?&//=]*)')
        x = pattern.sub(' ', x)
        x = url_pattern.sub('', x)
        x = x.strip()
        x = repeat_normalize(x, num_repeats=2)
        return x

    def encode(self, x, **kwargs):
        return self.tokenizer.encode(
            self.clean(str(x)),
            padding='max_length',
            max_length=self.hparams.max_length,
            truncation=True,
            #truncation => 설정한 문장 길이보다 긴 것은 잘라냄
            **kwargs,
        )

    def preprocess_dataframe(self, df):
        #위의 encode 매서드를 이용해 preprocessing함
        df['comment'] = df['comment'].map(self.encode)
        return df

    def dataloader(self, path, shuffle=False):
        #위의 2개의 매서드를 이용해 data loader 마무리
        df = self.read_data(path)
        df = self.preprocess_dataframe(df)

        dataset = TensorDataset(
            torch.tensor(df['comment'].to_list(), dtype=torch.long),
            torch.tensor(df['sentiment'].to_list(), dtype=torch.long),
        )
        return DataLoader(
            dataset,
            batch_size=self.hparams.batch_size * 1 if not self.hparams.tpu_cores else self.hparams.tpu_cores,
            shuffle=shuffle,
            num_workers=self.hparams.cpu_workers,
        )

    def train_dataloader(self):
        return self.dataloader(self.hparams.train_data_path, shuffle=True)

    def val_dataloader(self):
        return self.dataloader(self.hparams.val_data_path, shuffle=False)


In [ ]:
from pytorch_lightning.callbacks import ModelCheckpoint

checkpoint_callback = ModelCheckpoint(
    filename='epoch{epoch}-val_acc{val_acc:.4f}',
    monitor='val_acc',
    save_top_k=3,
    mode='max',
    auto_insert_metric_name=False,
)

https://colab.research.google.com/github/SurajDonthi/Multi-Camera-Person-Re-Identification/blob/master/demo.ipynb#scrollTo=HmFXmbJF5zPh 이 노트북에서 변경내용 찾아보기

In [ ]:
!nvidia-smi

Wed Dec  1 15:55:11 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.44       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   38C    P0    32W / 250W |   2285MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
print("Using PyTorch Ver", torch.__version__)
print("Fix Seed:", args['random_seed'])
seed_everything(args['random_seed'])
this_time = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
model = Model(**args)

print(":: Start Training ::")
trainer = Trainer(
    callbacks=[checkpoint_callback],
    max_epochs=args['epochs'],
    fast_dev_run=args['test_mode'],
    num_sanity_val_steps=None if args['test_mode'] else 0,
    # For GPU Setup
    deterministic=torch.cuda.is_available(),
    gpus=[0] if torch.cuda.is_available() else None,  # 0번 idx GPU  사용
    precision=16 if args['fp16'] and torch.cuda.is_available() else 32,
    # For TPU Setup
    # tpu_cores=args['tpu_cores'] if args['tpu_cores'] else None,
)
!rm -rf /content/runs/logger_file
!rm -rf /content/lightning_logs
writer = SummaryWriter('runs/logger_file')
#위 코드는 이전 실행으로 만들어진 log file 모두 삭제
#datetime과 같은 function을 이용해 파일 이름을 모두 다르게 했음으로 굳이 사용 안해도 됨
#사용 시에는 다시 한 번 logger_file 생성해야 함
trainer.fit(model)

In [ ]:
# %load_ext tensorboard
# %tensorboard --logdir=runs/logger_file/

In [ ]:
# !tensorboard dev upload --logdir=runs/logger_file \
#                         --name 'result_1'\
#                         --description 'Training results'\
#                         --one_shot\

위의 코드에 있던 self.log 부분을 따올 수 있는 지 확인 => ploting 하기

ckpt 및 log file을 drive로 옮기는 코드

In [ ]:
a = '/content/runs/logger_file/events.out.tfevents.1638374116.b80c5718ba2c.1478.8'
b = '/content/lightning_logs/version_0/checkpoints/epoch8-val_acc0.8100.ckpt'
name = 'Common_Kc'

copyfile(a, 
         '/content/drive/MyDrive/R&E_Data/'+name+a[32:])

copyfile(b, 
         '/content/drive/MyDrive/R&E_Data/'+name+'.ckpt')

latest_ckpt = sorted(glob('/content/drive/MyDrive/R&E_Data/'+name+'.ckpt'))[-1]
model = Model.load_from_checkpoint(latest_ckpt)

def infer(x):
    return torch.softmax(
        model(**model.tokenizer(x, return_tensors='pt')
    ).logits, dim=-1)

'/content/drive/MyDrive/R&E_Data/Common_Kc.ckpt'

--logdir에는 직접 folder path 작성하기<br>변수 사용금지!

Drive에 log file이 잘 올라갔는 지 확인하기 위해 작성함.

In [ ]:
# %load_ext tensorboard
# %tensorboard --logdir=/content/drive/MyDrive/R&E_Data/ --load_fast true

In [ ]:
latest_ckpt = sorted(glob('/content/drive/MyDrive/R&E_Data/nsmc_Kc.ckpt'))[-1]
7latest_ckpt

'/content/drive/MyDrive/R&E_Data/nsmc_Kc.ckpt'

In [ ]:
model = Model.load_from_checkpoint(latest_ckpt)

Downloading:   0%|          | 0.00/504 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/475M [00:00<?, ?B/s]

Some weights of the model checkpoint at beomi/KcELECTRA-base were not used when initializing ElectraForSequenceClassification: ['discriminator_predictions.dense_prediction.weight', 'discriminator_predictions.dense.weight', 'discriminator_predictions.dense.bias', 'discriminator_predictions.dense_prediction.bias']
- This IS expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at beomi/KcELECTRA-base and are newly initialized: ['classifier.out_proj.weight', 'classifi

Downloading:   0%|          | 0.00/288 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/387k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/124 [00:00<?, ?B/s]

torch.Tensor 형태로 반환된 데이터를 numpy ndarray로 변환함

id list내 csv file은 KcELECTRA에 넣었을 시, acc, mse error을 확인하는 코드

Checking Domain data accuracy

In [ ]:
id =  ['H_label',
      'E_label',
      'M_label',
      'N_label',
      'O_label',
      'S_label',
      'V&&C_label',
       'Common_label'
      ]
acc = np.array([], dtype=float)
for i in id:
    df = pd.read_csv('/content/'+i+'.csv')
    sen = df.sentiment.to_numpy(dtype=int)
    label = np.array([], dtype=int)
    loss = 0.0
    for j in df.comment:
        a = infer(j).cpu().detach().numpy()
        negative = a[:, 0]
        positive = a[:, 1]

        if positive > negative:
            loss += (1-positive)**2
            label = np.append(label, np.array([1]))
        else:
            loss += (0-positive)**2
            label = np.append(label, np.array([0]))            
    accuracy = accuracy_score(sen, label)
    print(i)
    print(accuracy)
    print(loss/2.)
    acc = np.append(acc, accuracy)

0.853
[37.698162]
H_label
0.707
[41.830387]
E_label
0.741
[44.0305]
M_label
0.827
[35.330345]
N_label
0.647
[49.518635]
O_label
0.744
[43.908844]
S_label
0.759
[43.054924]
V&&C_label


STUDENT MODEL

KcELECTRA

GPT-3

OpenAI doc 읽기 => 요청 및 메소드 사용 방법 / 학습 limit양 고려 / df.vlaues 형태로 데이터 전송

list 형식(또는 다른 형식도 가능)에서 Text를 뽑아내 str로 바꾸고 query로 보낸 후 response의 label만 따로 취해서 저장 => 활용

한글 인코딩 방법 또는 인덱스 이용해 query에 사용한 text 식별 가능하게 하기

https://beta.openai.com/docs/guides/classifications => score 값 따기

In [ ]:
# from transformers import GPT2TokenizerFast

# # Load the tokenizer.
# tokenizer = GPT2TokenizerFast.from_pretrained("gpt2")

# # Make sure the labels are formatted correctly.
# labels = ["positive", "negative", 'neutral']
# labels = [label.strip().lower().capitalize() for label in labels]

# # Encode the labels with extra white space prepended.
# labels_tokens = {label: tokenizer.encode(" " + label) for label in labels}
# print(labels_tokens)

Downloading:   0%|          | 0.00/0.99M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.29M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/665 [00:00<?, ?B/s]

{'Positive': [33733], 'Negative': [36183], 'Neutral': [25627]}


In [ ]:
# import os
# import openai

# l = ['yg오고나서 개성있는 스타일이 다 없어졌네',
#      '노래를 잘합니다.']
# data = list()
# cnt = 1

# openai.api_key = ''
# for i in l:
#     a = openai.Classification.create(
#              search_model='ada',
#              model='curie',
#              examples=[
#                        ['이번에도 질릴 줄 모르고 듣겠네요.', 'positive'],
#                        ['이번 앨범 대박이야 어떡해 악뮤는 늘 레전드다… 언제나 무슨 음악을 하든지 사랑해 이찬혁 이수현 악뮤❤️❤️', 'positive'],
#                        ['여기서 아이유 목소리 새롭게 들린다..ㅎ 저음 되게 좋다 악뮤랑도 목소리합 너무 좋은 듯', 'positive'],
#                        ['고음 부분이 조금...', 'negative'],
#                        ['찬혁아 저음 부분을 보강해야 할 것 같아', 'negative']
#              ],
#              query=str(i),
#              labels=['positive', 'negative', 'neutral'],
#              logprobs=3,
#              expand=['completion']    
#      )
#     data.append(a)
#     cnt += 1
# #positive == 1, negative == -1, neutral == 0

# for i in data:
#     print(i.label)
# #str type 의 결과(label)를 반환함

Negative
Positive


In [ ]:
# gpt_probs = pd.DataFrame(columns=['Positive', 'Negative', 'Neutral'])
# for i in data:
#     # Take the starting tokens for probability estimation.
#     # Labels should have distinct starting tokens.
#     # Here tokens are case-sensitive.
#     first_token_to_label = {tokens[0]: label for label, tokens in labels_tokens.items()}

#     top_logprobs = i["completion"]["choices"][0]["logprobs"]["top_logprobs"][0]
#     token_probs = {
#         tokenizer.encode(token)[0]: np.exp(logp) 
#         for token, logp in top_logprobs.items()
#     }
#     label_probs = {
#         first_token_to_label[token]: prob 
#         for token, prob in token_probs.items()
#         if token in first_token_to_label
#     }

#     # Fill in the probability for the special "Unknown" label.
#     if sum(label_probs.values()) < 1.0:
#         label_probs["Neutral"] = 1.0 - sum(label_probs.values())

#     gpt_probs = gpt_probs.append(label_probs, ignore_index=True)

위 코드에서 감정에 따른 각각의 key 값을 받아 pandas 형태로 만들기

In [ ]:
# label_probs

{'Negative': 0.19811145551870493,
 'Neutral': 0.018899706395704596,
 'Positive': 0.5649863711415478}

In [ ]:
# gpt_probs

,Positive,Negative,Neutral
0,0.306389,0.453070,0.029063
1,0.564986,0.198111,0.018900


Binary classification

In [ ]:
from transformers import GPT2TokenizerFast

# Load the tokenizer.
tokenizer = GPT2TokenizerFast.from_pretrained("gpt2")

# Make sure the labels are formatted correctly.
labels = ['negative', 'positive']
labels = [label.strip().lower().capitalize() for label in labels]

# Encode the labels with extra white space prepended.
labels_tokens = {label: tokenizer.encode(" " + label) for label in labels}
print(labels_tokens)

{'0': [657], '1': [352]}


In [ ]:
import os
import openai

l = ['yg오고나서 개성있는 스타일이 다 없어졌네',
     '노래를 잘합니다.']
data = list()
cnt = 1

openai.api_key = ''
for i in l:
    a = openai.Classification.create(
             search_model='ada',
             model='curie',
             examples=[
                       ['이번에도 질릴 줄 모르고 듣겠네요.', 'positive'],
                       ['이번 앨범 대박이야 어떡해 악뮤는 늘 레전드다… 언제나 무슨 음악을 하든지 사랑해 이찬혁 이수현 악뮤❤️❤️', 'positive'],
                       ['여기서 아이유 목소리 새롭게 들린다..ㅎ 저음 되게 좋다 악뮤랑도 목소리합 너무 좋은 듯', 'positive'],
                       ['고음 부분이 조금...', 'negative'],
                       ['찬혁아 저음 부분을 보강해야 할 것 같아', 'negative']
             ],
             query=str(i),
             labels=['positive', 'negative'],
             logprobs=3,
             expand=['completion']    
     )
    data.append(a)
    cnt += 1
#positive == 1, negative == -1, neutral == 0

for i in data:
    print(i.label)
#str type 의 결과(label)를 반환함

In [ ]:
gpt_probs = pd.DataFrame(columns=['Positive', 'Negative'])
for i in data:
    # Take the starting tokens for probability estimation.
    # Labels should have distinct starting tokens.
    # Here tokens are case-sensitive.
    first_token_to_label = {tokens[0]: label for label, tokens in labels_tokens.items()}

    top_logprobs = i["completion"]["choices"][0]["logprobs"]["top_logprobs"][0]
    token_probs = {
        tokenizer.encode(token)[0]: np.exp(logp) 
        for token, logp in top_logprobs.items()
    }
    label_probs = {
        first_token_to_label[token]: prob 
        for token, prob in token_probs.items()
        if token in first_token_to_label
    }

    # Fill in the probability for the special "Unknown" label.
    if sum(label_probs.values()) < 1.0:
        label_probs["Neutral"] = 1.0 - sum(label_probs.values())

    gpt_probs = gpt_probs.append(label_probs, ignore_index=True)

In [ ]:
label_probs

In [ ]:
gpt_probs

CLOVA_Sentiment

Json file 형식 변환과 학습 limit양 고려

In [ ]:
import json

In [ ]:
clova_content="재미없다"

clova_data = {
    "content": clova_content
}

print(json.dumps(clova_data, indent=4, sort_keys=True))
response = requests.post(naver_api_url, data=json.dumps(clova_data), headers=naver_headers)
rescode = response.status_code
if(rescode == 200):
    print(response.content)
else:
    print("Error : " + response.text)

r = bytes.decode(response.content)
result_naver = json.loads(r)

{
    "content": "\uc7ac\ubbf8\uc5c6\ub2e4"
}
b'{"document":{"sentiment":"negative","confidence":{"negative":99.969635,"positive":0.016905542,"neutral":0.013457015}},"sentences":[{"content":"\xec\x9e\xac\xeb\xaf\xb8\xec\x97\x86\xeb\x8b\xa4","offset":0,"length":4,"sentiment":"negative","confidence":{"negative":0.99817836,"positive":0.0010143325,"neutral":8.0742093E-4},"highlights":[{"offset":0,"length":4}]}]}'


In [ ]:
print(result_naver['document']['confidence'])
naver_probs = pd.DataFrame(columns=['negative', 'positive', 'neutral'])
naver_probs.append(result_naver['document']['confidence'], ignore_index=True)

{'negative': 99.969635, 'positive': 0.016905542, 'neutral': 0.013457015}


,negative,positive,neutral
0,99.969635,0.016906,0.013457
